In [1]:
# French Baby Names Map Visualization
import altair as alt
import pandas as pd
import geopandas as gpd

alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [2]:
names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)

names.sample(5)

,sexe,preusuel,annais,dpt,nombre
3573809,2,SOLENNE,1988,38,3
1219640,1,MOHAMED,1976,91,13
3420403,2,QUITTERIE,1996,33,5
2340128,2,ELODIE,1973,77,13
2881959,2,LIZA,2009,974,3


In [3]:
depts = gpd.read_file('departements-version-simplifiee.geojson')

depts.sample(5)

,code,nom,geometry
48,48,Lozère,"POLYGON ((3.36134 44.97141, 3.38637 44.95274, ..."
43,43,Haute-Loire,"POLYGON ((3.89741 45.35708, 3.91694 45.3394, 3..."
86,86,Vienne,"POLYGON ((-0.10212 47.0648, -0.09806 47.09135,..."
23,25,Doubs,"POLYGON ((6.80701 47.5628, 6.81666 47.54792, 6..."
31,31,Haute-Garonne,"POLYGON ((0.95398 43.78737, 0.9778 43.78644, 0..."


In [4]:
just_names = names

names = depts.merge(names, how='right', left_on='code', right_on='dpt')

In [5]:
chosen_name = 'Sacha'
chosen_department = 'Gard'

names['nombre'] = pd.to_numeric(names['nombre'], errors='coerce')
names['annais'] = pd.to_numeric(names['annais'], errors='coerce')

df = names.dropna(subset=['nombre', 'annais'])

df_name = df[df['preusuel'].str.upper() == chosen_name.upper()]

if chosen_department != 'All':
    df_name = df_name[df['nom'].str.upper() == chosen_department.upper()]

grouped = df_name.groupby(['annais', 'sexe'])['nombre'].sum().reset_index()

pivot = grouped.pivot(index='annais', columns='sexe', values='nombre').fillna(0)
pivot.columns = ['Boys', 'Girls']

pivot['Total'] = pivot['Girls'] + pivot['Boys']
pivot['% Girls'] = (pivot['Girls'] / pivot['Total']) * 100
pivot['% Boys'] = (pivot['Boys'] / pivot['Total']) * 100

plot_df = pivot[['% Girls', '% Boys']].reset_index().melt(id_vars='annais', 
                                                          var_name='Sex', 
                                                          value_name='Percentage')

plot_df['Sex'] = plot_df['Sex'].map({'% Girls': 'Girls', '% Boys': 'Boys'})

if chosen_department == 'All':
    title_str = f"Percentage of Boys and Girls Named {chosen_name} Over Time in France"
else:
    title_str = f"Percentage of Boys and Girls Named {chosen_name} Over Time in {chosen_department}"

chart = alt.Chart(plot_df).mark_line(point=True).encode(
    x=alt.X('annais:O', title='Year'),
    y=alt.Y('Percentage:Q', title='Percentage (%)'),
    color='Sex:N'
).properties(
    title=title_str,
    width=600,
    height=400
).interactive()

chart


c:\Users\medch\miniconda3\envs\SD\Lib\site-packages\geopandas\geodataframe.py:1750: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


alt.Chart(...)